In [1]:
import sys
import os
#connect local PATH


In [2]:
p = os.path.abspath(r'C:\Users\Ben\Desktop\Diplomatiki')
sys.path.append(str(p))

In [3]:
from CryptoSent.tools import utils as cs

In [4]:
import pandas as pd

In [5]:
s=pd.read_pickle(r'C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\Data Preparation\2_Data_Merging\submissions_2019__2021_06.pickle')

In [7]:
df=cs.submission_raw_processing(s)

In [45]:
# taking only crypto Subreddits
crypto_subreddits=['Bitcoin','CryptoCurrency','btc','CryptoMarkets','bitcoinbeginners',
                   'CryptoCurrencies','altcoin','icocrypto','CryptoCurrencyTrading','Crypto_General',
                   'ico','blockchain','ethereum','Ripple','litecoin','Monero','Stellar','CryptoCurrencyClassic']
df=df[df.subreddit.isin(crypto_subreddits)]

In [125]:
#extracting awards
ts=pd.concat([(df[df.all_awardings=='Empty'].iloc[0]),
           (df[(df.all_awardings.apply(lambda x: len(x))==1)&(df.all_awardings!='Empty')].iloc[1]),
           (df[(df.all_awardings.apply(lambda x: len(x))>2)&(df.all_awardings!='Empty')].iloc[1])]
           ,axis=1).T

In [126]:
ts.all_awardings.apply()

,author,all_awardings,num_comments,score,title,selftext,subreddit,subreddit_subscribers,id,domain,no_follow,send_replies,author_fullname,subreddit_id,permalink,url,created,author_created
2019-01-01 01:00:29,[deleted],Empty,0,1,DENT Exchange live. Time to recognize DENT!,[removed],CryptoCurrency,854254,abcuni,self.CryptoCurrency,True,True,NaN,t5_2wlj3,/r/CryptoCurrency/comments/abcuni/dent_exchang...,https://www.reddit.com/r/CryptoCurrency/commen...,2019-01-01 01:00:29,NaT
2019-03-01 11:59:37,Block_Venture,"[[Gold, Gold Award, 1, 500, 100]]",5,5,"Chainstarter - Nick and Obadiah Ayton, the pen...",NaN,CryptoCurrency,885702,aw3l6f,linkedin.com,False,True,t2_9tb0ktv,t5_2wlj3,/r/CryptoCurrency/comments/aw3l6f/chainstarter...,https://www.linkedin.com/pulse/chainstarter-fa...,2019-03-01 11:59:37,2017-08-08 17:59:51
2019-03-10 17:59:47,nullc,"[[Platinum, Platinum Award, 1, 1800, 0], [Gold...",154,41,"Don't just blame the monster, blame his makers",NaN,btc,254418,azhnuo,0bin.net,False,False,t2_1y2aw,t5_2si5v,/r/btc/comments/azhnuo/dont_just_blame_the_mon...,https://0bin.net/paste/0Hqu2j9q+CyawdOx#ONzZK8...,2019-03-10 17:59:47,2007-06-13 13:52:20


In [104]:
s.all_awardings.apply(lambda x: list(zip(*x))[0])

created
2019-03-04 18:23:40                             (Platinum, Gold, Silver)
2019-03-10 17:59:47                             (Platinum, Gold, Silver)
2019-03-14 21:41:25                             (Platinum, Gold, Silver)
2019-04-08 16:06:08                             (Platinum, Gold, Silver)
2019-04-11 23:50:22                             (Platinum, Gold, Silver)
                                             ...                        
2021-06-30 22:20:05    (Timeless Beauty, Helpful (Pro), Gold, Bravo!,...
2021-06-30 23:33:35                            (Wholesome, Silver, Hugz)
2021-06-30 23:57:02    (All-Seeing Upvote, Wholesome, Silver, Hugz, T...
2021-07-01 00:13:41    (All-Seeing Upvote, Platinum, To The Stars, He...
2021-07-01 01:22:14      (Helpful, Wholesome, Silver, Masterpiece, Hugz)
Name: all_awardings, Length: 9088, dtype: object